In [1]:
from utils import *


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rd81\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_df_original, val_df_original = get_processed_data(dev=False)


In [5]:
train_df_original

,propaganda,snippet_label,original,original_sentence_no_tags,snippet_original,sentence_tokenised_no_tags,snippet_tokenised
0,0,not_propaganda,"No, <BOS> he <EOS> will not be confirmed.","No, he will not be confirmed.",he,"[No, ,, he, will, not, be, confirmed, .]",[he]
1,0,not_propaganda,This declassification effort <BOS> won’t make ...,This declassification effort won’t make thing...,won’t make things any worse than they are for ...,"[This, declassification, effort, won, ’, t, ma...","[won, ’, t, make, things, any, worse, than, th..."
2,1,flag_waving,"""The Obama administration misled the <BOS> Ame...","""The Obama administration misled the American...",American people,"[``, The, Obama, administration, misled, the, ...","[American, people]"
3,0,not_propaganda,“It looks like we’re capturing the demise of t...,“It looks like we’re capturing the demise of t...,and,"[“, It, looks, like, we, ’, re, capturing, the...",[and]
4,0,not_propaganda,"<BOS> Location: Westerville, Ohio <EOS>","Location: Westerville, Ohio","Location: Westerville, Ohio","[Location, :, Westerville, ,, Ohio]","[Location, :, Westerville, ,, Ohio]"
...,...,...,...,...,...,...,...
2555,0,not_propaganda,<BOS> We support and appreciate <EOS> your bus...,We support and appreciate your business.”,We support and appreciate,"[We, support, and, appreciate, your, business,...","[We, support, and, appreciate]"
2556,0,not_propaganda,International Atomic Energy Agency (IAEA) Dire...,International Atomic Energy Agency (IAEA) Dire...,capacity to check whether Iran was conducting ...,"[International, Atomic, Energy, Agency, (, IAE...","[capacity, to, check, whether, Iran, was, cond..."
2557,0,not_propaganda,What has been done: there has been work on for...,What has been done: there has been work on for...,one for those recently,"[What, has, been, done, :, there, has, been, w...","[one, for, those, recently]"
2558,0,not_propaganda,This is <BOS> the law of gradualness not the g...,This is the law of gradualness not the gradua...,the law of gradualness not the gradualness of ...,"[This, is, the, law, of, gradualness, not, the...","[the, law, of, gradualness, not, the, gradualn..."


In [6]:
class CustomPropagandaDataset_vanilla(Dataset):
    def __init__(self,df,dataset='prop'):
        
        tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
        
        if dataset == 'prop':
            target_col = 'propaganda'
            text_col = 'original_sentence_no_tags'
            
            self.labels=torch.tensor([label for label in df[target_col]])

            
        if dataset == 'snip':
            label_to_id = {'flag_waving': 0, 'exaggeration,minimisation': 1, 'causal_oversimplification': 2, 'name_calling,labeling': 3, 'repetition': 4, 'doubt': 5, 'not_propaganda': 6, 'loaded_language': 7, 'appeal_to_fear_prejudice': 8}
            
            target_col = 'snippet_label'
            text_col = 'snippet_original'
            self.labels=torch.tensor([label_to_id[label] for label in df[target_col]])
            
        self.texts=[tokenizer(text,padding='max_length',max_length=200,truncation=True,return_tensors="pt") for text in df[text_col]]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self,idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self,idx):
        return self.texts[idx]

    def __getitem__(self,idx):
        batch_texts=self.get_batch_texts(idx)
        batch_y=self.get_batch_labels(idx)

        return batch_texts,batch_y


def prepare_inputs(input1,label,device):
  label=label.to(device)
  mask=input1['attention_mask'].to(device)
  input_id=input1['input_ids'].squeeze(1).to(device)
  return (input_id,mask,label)

In [7]:
train_data = CustomPropagandaDataset_vanilla(train_df_original)
val_data = CustomPropagandaDataset_vanilla(val_df_original)

train_dataloader=torch.utils.data.DataLoader(train_data,batch_size=64,shuffle=True)
val_dataloader=torch.utils.data.DataLoader(val_data,batch_size=64)


In [16]:
train_data[0][1]

array(0, dtype=int64)

In [27]:
for i in train_dataloader:
    print(len(i[1]))
    break

64
